Длинные названия потому, что мне лень писать документацию, а за жизненным циклом обектов следить не умею :)

![image](./pic1.jpg)
![image](./pic2.jpg)

In [1]:
Generate_Network_t0 <- function(size = 40, sparsity = 0.7) {
  network <- matrix(0, nrow=size, ncol=size)
  
  for (i in 1:size) {
    for (j in 1:size) {
      if (runif(1) > sparsity) {
        network[i, j] <- 1
      }
    }
  }
  
  return(network)
}


# Смулирую ТОЛЬКО возникновение сети


Network_function_1 <- function(i, Network) { # The out-degree (density) | Зачем тут минишаг? Для ковариат сложнее?
  net <- 0
  for (j in 1:length(Network[i, ])) {
    net <- net + Network[i, j]
  }
  return(net)
}

Network_function_3 <- function(i, Network) { # The transTrip
  net <- 0
  for (j in 1:length(Network[i, ])) {
      for(h in 1:length(Network[i, ])){
          net <- net + (Network[i,j]*Network[i,h]*Network[h,j])
      }
  }
  return(net)
}

Generate_Possible_Ministep_Matrix_vector <- function(i, Network) {
  Networks <- list()
  
  for (j in 1:length(Network[i, ])) {
    if (i != j) {
      TMP_Net <- Network
      TMP_Net[i, j] <- 1
      Networks[[j]] <- TMP_Net    
    }
  }
  return(Networks)
}

Ministep <- function(Network, i, j) {
  Network[i, j] <- 1
  
  return(Network)
}

Distribution_actors <- function(i, Network, beta) {
  Util_i_j <- 0
  Util <- 0
  for (j in 1:length(Network[i, ])) {

        Util_i_j[j] <- Utility(Network, beta, i, j)

        Util_i_h <- 0
        for (h in 1:length(Network[i, ])) {
          if (j != h) {
            Util_i_h <- Util_i_h + Utility(Network, beta, i, h)
          }
        }
        
        Util[j] <- Util_i_j[j] / Util_i_h
      
        Util[i] <- 0

  }
  return(Util)
}


Utility <- function(Network, beta, i, j) {
  util <- 0
  util <- util + (beta[1] * Network_function_1(i, Network)) + (beta[2] * Network_function_3(i, Network)) # Если есть еще функции добавлять через сумму
  
  return(util)
}


Simulator <- function(Network0, t = 0, LambdaV, total_Lambda, T, beta) {
  Network <- Network0 
  while (t < T) {
    deltaT <- rexp(1, rate=total_Lambda)
    i <- sample(1:length(Network0[1, ]), size=1, prob=(LambdaV / total_Lambda))
    j <- sample(1:length(Network0[, 1]), size=1, prob=Distribution_actors(i, Network, beta)) # Тут надо задать распределение
    Network[i, j] <- 1
    t <- t + deltaT
  }
  return(Network)
}


In [2]:
size <- 15
Network0 <- Generate_Network_t0(size = size)
LambdaV <- runif(n = size, min = 0.1, max = 5) 
total_Lambda <- sum(LambdaV)
total_Lambda
beta <- c(2, 6) 
T <- 2

[1] 43.00097

In [3]:
write.table(Network0, file = "Network0.dat", sep = " ", row.names = FALSE, col.names = FALSE)

In [4]:
Network1 <- Simulator(Network0, t = 0, LambdaV, total_Lambda, T, beta)

In [5]:
write.table(Network1, file = "Network1.dat", sep = " ", row.names = FALSE, col.names = FALSE)

In [6]:
Network2 <- Simulator(Network1, t = 0, LambdaV, total_Lambda, T, beta)

In [7]:
write.table(Network2, file = "Network2.dat", sep = " ", row.names = FALSE, col.names = FALSE)

In [8]:
sum(Network0)
sum(Network1)
sum(Network2)

[1] 66

[1] 119

[1] 147

---
# Проверка в RSiena

In [9]:
library(RSiena)

In [10]:



Networks <- array(c(Network0, Network1, Network2), dim = c(size, size, 3))

Networks <- sienaDependent(Networks, nodeSet="Actors")

Networks
     

Type         oneMode             
Observations 3                   
Nodeset      Actors (15 elements)


In [11]:
mydata <- sienaDataCreate(Networks)
myeff <- getEffects(mydata)
mydata

For dependent variable Networks, in some periods,

there are only increases, or only decreases.

This will be respected in the simulations. 

If this is not desired, use allowOnly=FALSE when creating the dependent variable.



Dependent variables:  Networks 
Number of observations: 3 

Nodeset                  Actors 
Number of nodes              15 

Dependent variable Networks      
Type               oneMode       
Observations       3             
Nodeset            Actors        
Densities          0.29 0.54 0.68

"uponly":  Networks:    all periods

In [12]:
# myeff <- includeEffects(myeff, outPop, type = "creation", include =TRUE)
# myeff <- includeEffects(myeff, outRate, type = "rate", include =TRUE)
# myeff <- includeEffects(myeff, outRateLog, type = "rate", include = TRUE)

# myeff <- includeEffects(myeff, transTrip, type = "creation", include = TRUE)
myeff <- includeEffects(myeff, transTrip, include = TRUE)

  effectName          include fix   test  initialValue parm
1 transitive triplets TRUE    FALSE FALSE          0   0   


In [13]:
summary(myeff)

,name,effectName,functionName,shortName,interaction1,interaction2,type,basicRate,include,randomEffects,⋯,effect3,interactionType,local,setting,effectFn,statisticFn,netType,groupName,group,effectNumber
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,⋯,<int>,<chr>,<lgl>,<chr>,<list>,<list>,<chr>,<chr>,<dbl>,<int>
Networks.rate.1,Networks,constant Networks rate (period 1),Amount of network change in period 1,Rate,,,rate,TRUE,TRUE,FALSE,⋯,0,,TRUE,,NULL,NULL,oneMode,Group1,1,1
Networks.rate.2,Networks,constant Networks rate (period 2),Amount of network change in period 2,Rate,,,rate,TRUE,TRUE,FALSE,⋯,0,,TRUE,,NULL,NULL,oneMode,Group1,1,2
Networks.obj.eval.1,Networks,reciprocity,Number of reciprocated ties,recip,,,eval,FALSE,TRUE,FALSE,⋯,0,dyadic,TRUE,,NULL,NULL,oneMode,Group1,1,12
Networks.obj.eval.2,Networks,transitive triplets,Number of transitive triplets,transTrip,,,eval,FALSE,TRUE,FALSE,⋯,0,,TRUE,,NULL,NULL,oneMode,Group1,1,18


In [14]:
myalgorithm <- sienaAlgorithmCreate(projname = 'Тест симулятора сети')
ans <- siena07( myalgorithm, data = mydata, effects = myeff, clusterType="FORK", useCluster=TRUE, nbrNodes=12)

If you use this algorithm object, siena07 will create/use an output file Тест симулятора сети.txt .


In [15]:
ans

Estimates, standard errors and convergence t-ratios

                                   Estimate   Standard   Convergence 
                                                Error      t-ratio   

Rate parameters: 
  0.1      Rate parameter period 1 3.9417   ( 0.5551   )             
  0.2      Rate parameter period 2 2.1897   ( 0.4283   )             

Other parameters: 
  1.  eval reciprocity             0.3977   ( 0.2866   )   0.0158    
  2.  eval transitive triplets     0.0152   ( 0.0835   )   0.0596    

Overall maximum convergence ratio:    0.0611 


Total of 1673 iteration steps.


Параметры изначальные

Необходимо почнимть, что оцениваем $ exp(\text{парамтер}) $ (Для созданных мной параметров)

In [17]:
total_Lambda
log(total_Lambda)

log(beta)


[1] 43.00097

[1] 3.761223

[1] 0.6931472 1.7917595